In [1]:
import sys
import os
import json

import pandas as pd
import numpy as np

from collections import defaultdict

import psycopg2
import psycopg2.extras as extras
from psycopg2 import extensions

import sqlalchemy
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

In [2]:
root_dir = 'C:/Users/delst/OneDrive/Desktop/Code/Workspace/JSON_to_PGSQL'
sys.path.append(root_dir)

from A_Main.Configurations.setup_env import setup_environment
config = setup_environment(root_dir)

In [3]:
with open(config.postgres_password, 'r') as file:
    db_pw = file.read()

---

In [4]:
conn = psycopg2.connect(
    dbname="json_to_pgsql",
    user="postgres",
    password=db_pw,
    host="localhost"
)

In [5]:
cur = conn.cursor()

In [6]:
cur.execute("SELECT * FROM post_data;")
post_data_results = cur.fetchall()
column_names = [desc[0] for desc in cur.description]

df_post_data = pd.DataFrame(post_data_results, columns=column_names)
df_post_data.head()

,index,allow_live_comments,author_patreon_flair,secure_media,upvote_ratio,author_flair_css_class,link_flair_template_id,is_meta,contest_mode,banned_by,...,quarantine,thumbnail_width,gildings,send_replies,link_flair_text,can_gild,author_flair_template_id,removed_by,num_comments,approved_by
0,0,True,False,None,0.96,None,2491e734-cd17-11ed-9c62-06fbff1c98dd,False,False,None,...,False,140.0,{},True,Clubhouse,True,None,None,1221,None
1,1,False,False,None,0.96,None,62cc9266-f2b0-11ec-9096-baad01c86c30,False,False,None,...,False,140.0,{},True,❔ Other,True,None,None,477,None
2,2,True,False,"{'reddit_video': {'bitrate_kbps': 1200, 'dash_...",0.95,None,5e058b84-4acb-11ed-ae2b-6a59e86b4614,False,False,None,...,False,140.0,{},True,Video/Gif,True,None,None,1704,None
3,3,True,False,None,0.96,None,None,False,False,None,...,False,140.0,{},True,None,True,None,None,611,None
4,4,False,False,"{'reddit_video': {'bitrate_kbps': 2400, 'dash_...",0.91,None,b16ae4c8-c28e-11ed-9954-2e95b348321f,False,False,None,...,False,140.0,{},True,Science,True,None,None,848,None


In [7]:
cur.execute("SELECT * FROM video_data;")
video_data_results = cur.fetchall()
column_names = [desc[0] for desc in cur.description]

df_video_data = pd.DataFrame(video_data_results, columns=column_names)
df_video_data.head()

,index,post_id,bitrate_kbps,dash_url,duration,fallback_url,has_audio,height,hls_url,is_gif,scrubber_media_url,transcoding_status,width
0,0,149yrlt,1200,https://v.redd.it/evmp79fbr56b1/DASHPlaylist.m...,68,https://v.redd.it/evmp79fbr56b1/DASH_480.mp4?s...,True,480,https://v.redd.it/evmp79fbr56b1/HLSPlaylist.m3...,False,https://v.redd.it/evmp79fbr56b1/DASH_96.mp4,completed,854
1,1,149z3aw,2400,https://v.redd.it/9az2zw4ou56b1/DASHPlaylist.m...,58,https://v.redd.it/9az2zw4ou56b1/DASH_720.mp4?s...,None,720,https://v.redd.it/9az2zw4ou56b1/HLSPlaylist.m3...,False,https://v.redd.it/9az2zw4ou56b1/DASH_96.mp4,completed,406
2,2,149wwzb,2400,https://v.redd.it/vstcrxxm756b1/DASHPlaylist.m...,178,https://v.redd.it/vstcrxxm756b1/DASH_720.mp4?s...,None,720,https://v.redd.it/vstcrxxm756b1/HLSPlaylist.m3...,False,https://v.redd.it/vstcrxxm756b1/DASH_96.mp4,completed,406
3,3,14a36xk,2400,https://v.redd.it/regbdl8au66b1/DASHPlaylist.m...,43,https://v.redd.it/regbdl8au66b1/DASH_720.mp4?s...,False,720,https://v.redd.it/regbdl8au66b1/HLSPlaylist.m3...,True,https://v.redd.it/regbdl8au66b1/DASH_96.mp4,completed,1280
4,4,149zpnd,1200,https://v.redd.it/0zpvfvyk066b1/DASHPlaylist.m...,354,https://v.redd.it/0zpvfvyk066b1/DASH_480.mp4?s...,True,394,https://v.redd.it/0zpvfvyk066b1/HLSPlaylist.m3...,False,https://v.redd.it/0zpvfvyk066b1/DASH_96.mp4,completed,854


In [8]:
cur.execute("SELECT * FROM image_data;")
image_data_results = cur.fetchall()
column_names = [desc[0] for desc in cur.description]

df_image_data = pd.DataFrame(image_data_results, columns=column_names)
df_image_data

,index,post_id,id,variants
0,0,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,"{'nsfw': None, 'obfuscated': None}"
1,1,14a1hq5,gAk-dl1pjVUL88HkOOUMiT7PqP6wZncsweXFa-i6fUI,"{'nsfw': None, 'obfuscated': None}"
2,2,149yrlt,NndmejhuYmJyNTZiMb0UmIeULhm5TqEhpHt9vw3gmBGF49...,"{'nsfw': None, 'obfuscated': None}"
3,3,14a064k,YQCgKRT3PwoUJd7UixJDP4_jcfVx1QEnV0W2TQH8uPo,"{'nsfw': None, 'obfuscated': None}"
4,4,149z3aw,_UjbEUYm4O61zhRgxj2pPo4GfL-JF2P5N7mdZq9pNTE,"{'nsfw': None, 'obfuscated': None}"
5,5,149z5hi,s9cucyfhWN9ulXzMU3x5IUxt27QjyGSuT2BHB-fhtHo,"{'nsfw': None, 'obfuscated': None}"
6,6,149zmar,WB-HQNbjw_jdM0sDWn5YwZGVQxga-2dnwtT8INDq0SI,"{'nsfw': None, 'obfuscated': None}"
7,7,149wwzb,OWw3ZGthdW03NTZiMULWJPqfSSTCh65EjnIKh6SILam-gV...,"{'nsfw': None, 'obfuscated': None}"
8,8,149xfsx,uR11yEZ0zz2GqGsaxBNGGiYfaty5aiYxkQh55o4RN1E,"{'nsfw': None, 'obfuscated': None}"
9,9,14a36xk,pkNUa4Sb1R67JVfTAmcKCqTegLfnfl9KMbXY_0wJCeM,"{'nsfw': None, 'obfuscated': None}"


In [9]:
cur.execute("SELECT * FROM image_resolution;")

resolution_data_results = cur.fetchall()
column_names = [desc[0] for desc in cur.description]

df_resolution_data = pd.DataFrame(resolution_data_results, columns=column_names)
df_resolution_data

,index,post_id,id,height,url,width
0,0,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,171,https://preview.redd.it/urxp0hrc666b1.png?widt...,108
1,1,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,343,https://preview.redd.it/urxp0hrc666b1.png?widt...,216
2,2,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,508,https://preview.redd.it/urxp0hrc666b1.png?widt...,320
3,3,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,1016,https://preview.redd.it/urxp0hrc666b1.png?widt...,640
4,4,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,1524,https://preview.redd.it/urxp0hrc666b1.png?widt...,960
...,...,...,...,...,...,...
102,102,149vuvp,dlXFVWswaCF6GooP3Nh-vUvJ3wacL0XCyygdkQ4l_Zo,193,https://preview.redd.it/il6cqa38w46b1.jpg?widt...,216
103,103,149vuvp,dlXFVWswaCF6GooP3Nh-vUvJ3wacL0XCyygdkQ4l_Zo,287,https://preview.redd.it/il6cqa38w46b1.jpg?widt...,320
104,104,149vuvp,dlXFVWswaCF6GooP3Nh-vUvJ3wacL0XCyygdkQ4l_Zo,574,https://preview.redd.it/il6cqa38w46b1.jpg?widt...,640
105,105,149vuvp,dlXFVWswaCF6GooP3Nh-vUvJ3wacL0XCyygdkQ4l_Zo,862,https://preview.redd.it/il6cqa38w46b1.jpg?widt...,960


---

In [10]:
cur.execute("SELECT table_name, column_name, data_type FROM information_schema.columns WHERE table_schema = 'public' ORDER BY table_name, ordinal_position;")
get_dtypes = cur.fetchall()
column_names = [desc[0] for desc in cur.description]

df_get_dtypes = pd.DataFrame(get_dtypes, columns=column_names)
df_get_dtypes

,table_name,column_name,data_type
0,image_data,index,bigint
1,image_data,post_id,text
2,image_data,id,text
3,image_data,variants,text
4,image_resolution,index,bigint
...,...,...,...
252,video_data,hls_url,text
253,video_data,is_gif,boolean
254,video_data,scrubber_media_url,text
255,video_data,transcoding_status,text


In [11]:
df_image_data.dtypes

index        int64
post_id     object
id          object
variants    object
dtype: object

---

In [12]:
df_res = df_resolution_data
df_res.head()

,index,post_id,id,height,url,width
0,0,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,171,https://preview.redd.it/urxp0hrc666b1.png?widt...,108
1,1,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,343,https://preview.redd.it/urxp0hrc666b1.png?widt...,216
2,2,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,508,https://preview.redd.it/urxp0hrc666b1.png?widt...,320
3,3,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,1016,https://preview.redd.it/urxp0hrc666b1.png?widt...,640
4,4,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,1524,https://preview.redd.it/urxp0hrc666b1.png?widt...,960


In [13]:
# Optimised NumPy computations using vectorization, useful for large datasets
pixels = (df_res['height'].to_numpy() * df_res['width'].to_numpy()).tolist()
pixels

[18468,
 74088,
 162560,
 650240,
 1463040,
 14472,
 58104,
 127680,
 511360,
 1150080,
 6480,
 26136,
 57600,
 230400,
 518400,
 7776,
 31104,
 68160,
 272640,
 614400,
 777600,
 20736,
 82944,
 181760,
 13284,
 53352,
 117120,
 468480,
 8748,
 34992,
 76800,
 307200,
 691200,
 874800,
 20736,
 82944,
 182080,
 728320,
 11664,
 46656,
 102400,
 409600,
 921600,
 1166400,
 6480,
 26136,
 57600,
 230400,
 518400,
 655560,
 5292,
 21384,
 47360,
 189440,
 426240,
 538920,
 8640,
 34776,
 76480,
 306560,
 690240,
 873720,
 9180,
 36720,
 80640,
 20736,
 82944,
 182080,
 728320,
 1638720,
 7668,
 30672,
 67520,
 270720,
 608640,
 771120,
 20520,
 82296,
 180800,
 5724,
 23112,
 50880,
 204160,
 13500,
 54216,
 119040,
 476800,
 11448,
 45792,
 100480,
 402560,
 6048,
 24408,
 53440,
 214400,
 20736,
 82944,
 181760,
 727680,
 1637760,
 2073600,
 10368,
 41688,
 91840,
 367360,
 827520,
 1046520]

---

In [14]:
cur.execute("SELECT ups, num_comments, ups / num_comments AS comment_ratio FROM post_data;")
dummy_table = cur.fetchall()

column_names = [desc[0] for desc in cur.description]

df_dummy = pd.DataFrame(dummy_table, columns=column_names)
df_dummy

,ups,num_comments,comment_ratio
0,12836,1221,10
1,8493,477,17
2,27260,1704,15
3,10583,611,17
4,14805,848,17
5,11917,184,64
6,8773,1631,5
7,19966,1737,11
8,18318,213,86
9,4147,206,20
